In [4]:
import requests
from bs4 import BeautifulSoup

# You can get this block to work by getting the CURL as shown in the video but otherwise just use the database to run the sqlite functions below. I did not supply it because it contains personal information pertaining to my computer
# This block is scrapping imdb for the html for the top 25 movies on imdb (They don't like large requests or commercial scrapping)
cookies = {
}

headers = {
}

response = requests.get('https://www.imdb.com/chart/top/', cookies=cookies, headers=headers)


In [5]:
# This block is identifying and isolationg the information I want from the response
soup = BeautifulSoup(response.text, 'html.parser')

# These are the 3 sections with the data I want
titles = soup.findAll("h3", {"class": "ipc-title__text"})
year_leng_agerats = soup.findAll("span", {"class": "sc-b189961a-8 kLaxqf cli-title-metadata-item"})
viewerrat_reviewcounts = soup.findAll("span", {"class": "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating"})

# 'titles' is indexed and I want just the movie title so this removes the index (ex. '23. Forest Gump' -> 'Forest Gump')
def keep_after_dot(data):
    dot_index = data.find(". ")
    if dot_index != -1:
        return data[dot_index + 2:]
    else:
        return data

# As the name suggests, year_leng_agerats is the year, movie length, and age rating but they are separate in the html so this is joining them (ex. '1967', '1h 23m', 'R', '1988', '2h 3m' ...) '^' is a delimeter for later
def combine_every_three(data):
    combined_list = []
    for i in range(0, len(data), 3):
        combined_list.append("^".join(data[i:i+3]))
    return combined_list

# viewerrat_reviewcounts is the combination of viewer rating and review count and there are '()' that are inconsistent with the rest of the data I'm using so I'm removing them now. '^' is a delimiter for later
def split_remove_par(data):
    data = data.replace("(", "").replace(")", "")
    parts = data.split()
    parts[1] = str(convert_to_number(parts[1]))
    formatted_text = "^".join(parts)
    return formatted_text

# Review Count is a string '1.9M' so I'm converting it to a float
def convert_to_number(data):
    # Check if the text ends with 'M' for millions
    if data.endswith('M'):
        # Remove the 'M' and convert the remaining part to a float
        number = float(data[:-1])
        # Multiply by 1,000,000 to get the full number
        return int(number * 1000000)
    # Check if the text ends with 'K' for thousands
    elif data.endswith('K'):
        # Remove the 'K' and convert the remaining part to a float
        number = float(data[:-1])
        # Multiply by 1,000 to get the full number
        return int(number * 1000)
    else:
        # If no 'M' or 'K', return the original text converted to a number
        return int(float(data))

# This function converts the movie length from '1h 5m' to 65 which will be in the database as minutes so I can compare data as integers instead of a string
def convert_to_minutes(lengths):
    minutes_list = []
    for duration in lengths:
        parts = duration.split()
        total_minutes = 0
        for part in parts:
            if part.endswith('h'):
                total_minutes += int(part[:-1]) * 60
            elif part.endswith('m'):
                total_minutes += int(part[:-1])
        minutes_list.append(total_minutes)
    return minutes_list


# Combine the data
year_leng_agerats_fixed = combine_every_three([x.text for x in year_leng_agerats])
viewerrat_reviewcounts_fixed = [split_remove_par(view.text) for view in viewerrat_reviewcounts]

# These will be the columns in the database, titles is already alone and the other 5 lists will be populated in the for loops below
titles = [keep_after_dot(title.text) for title in titles][1:]
releases = []
lengths = []
age_ratings = []
scores = []
review_counts = []

# Splitting year_leng_agerats_fixed into individual columns and appending to their list
for row in year_leng_agerats_fixed:
    parts = row.split('^')
    releases.append(int(parts[0]))
    lengths.append(parts[1])
    age_ratings.append(parts[2])

# Splitting viewerrat_reviewcounts_fixed into individual columns and appending to their list
for row in viewerrat_reviewcounts_fixed:
    parts = row.split('^')
    scores.append(float(parts[0]))
    review_counts.append(int(parts[1]))

lengths = convert_to_minutes(lengths)

# zipping together for easy table creation
movies = zip(titles, releases, lengths, age_ratings, scores, review_counts)

In [6]:
import sqlite3
def insert_into_database(movies):
    # Connect to SQLite database (create it if not exists)
    conn = sqlite3.connect('movies_data.db')
    c = conn.cursor()
    
    # Create table if not exists
    c.execute('''CREATE TABLE IF NOT EXISTS movie_data
                 (title TEXT, release REAL, length_min REAL, age_rating TEXT, score REAL, review_count REAL)''')
    
    
    # Insert data into the table
    c.executemany("INSERT INTO movie_data VALUES (?, ?, ?, ?, ?, ?)", movies)
    
    # Commit changes and close connection
    conn.commit()
    conn.close()

insert_into_database(movies)

In [7]:
def insert_row(title, release, length_min, age_rating, score, review_count):
    # Connect to SQLite database
    conn = sqlite3.connect('movies_data.db')
    c = conn.cursor()
    
    # Define the data to be inserted
    data = (title, release, length_min, age_rating, score, review_count)
    
    # Insert data into the table
    c.execute("INSERT INTO movie_data VALUES (?, ?, ?, ?, ?, ?)", data)
    
    # Commit changes and close connection
    conn.commit()
    conn.close()
    
# Example data for the new row
title = "New Movie"
release = 2023
length_min = 120
age_rating = "PG-13"
score = 8.7
review_count = 50000

# Insert the new row into the SQLite table
insert_row(title, release, length_min, age_rating, score, review_count)

In [8]:
def update_movie_title(new_title, old_title):
    # Connect to SQLite database
    conn = sqlite3.connect('movies_data.db')
    c = conn.cursor()
    
    # Define the new title and old title
    new_title = new_title
    old_title = old_title
    
    # Update the movie title in the table
    c.execute("UPDATE movie_data SET title = ? WHERE title = ?", (new_title, old_title))
    
    # Commit changes and close connection
    conn.commit()
    conn.close()

# Example new title
new_title = "New Movie 2"
old_title = "New Movie"

# Update the movie title in the SQLite table
update_movie_title(new_title, old_title)

In [9]:
def retrieve_movies_by_title(title):
    # Connect to SQLite database
    conn = sqlite3.connect('movies_data.db')
    c = conn.cursor()
    
    # Execute a SELECT query with a WHERE clause
    c.execute("SELECT * FROM movie_data WHERE title = ?", (title,))
    
    # Fetch all rows that match the condition
    rows = c.fetchall()
    
    # Print or process the retrieved rows
    for row in rows:
        print(row)  # Print each row for demonstration
    
    # Close connection
    conn.close()

# Example title to search for
title_to_search = "New Movie 2"

# Retrieve rows with the specified title from the SQLite table
retrieve_movies_by_title(title_to_search)

('New Movie 2', 2023.0, 120.0, 'PG-13', 8.7, 50000.0)


In [10]:
def delete_movies_by_title(title):
    # Connect to SQLite database
    conn = sqlite3.connect('movies_data.db')
    c = conn.cursor()
    
    # Execute a DELETE query with a WHERE clause
    c.execute("DELETE FROM movie_data WHERE title = ?", (title,))
    
    # Commit changes to the database
    conn.commit()
    
    # Close connection
    conn.close()

# Example title to delete
title_to_delete = "New Movie 2"

# Delete rows with the specified title from the SQLite table
delete_movies_by_title(title_to_delete)

In [11]:

def get_average_release_year():
    # Connect to SQLite database
    conn = sqlite3.connect('movies_data.db')
    c = conn.cursor()
    
    # Execute a SELECT query to calculate the average release year
    c.execute("SELECT AVG(release) FROM movie_data")
    
    # Fetch the result
    avg_year = c.fetchone()[0]
    
    # Close connection
    conn.close()
    
    return avg_year

# Get the average release year from the SQLite table
average_release_year = get_average_release_year()

print(f"The average release year is: {int(average_release_year)}")

The average release year is: 1988


In [12]:
def get_age_rating_counts():
    # Connect to SQLite database
    conn = sqlite3.connect('movies_data.db')
    c = conn.cursor()
    
    # Execute a SELECT query to group by age_rating and count occurrences
    c.execute("SELECT age_rating, COUNT(*) FROM movie_data GROUP BY age_rating")
    
    # Fetch all results
    age_rating_counts = c.fetchall()
    
    # Close connection
    conn.close()
    
    return age_rating_counts

# Get the counts of age ratings from the SQLite table
age_rating_counts = get_age_rating_counts()

# Print the results
print("Age Rating Counts:")
for age_rating, count in age_rating_counts:
    print(f"{age_rating}: {count}")

Age Rating Counts:
Approved: 1
Not Rated: 1
PG: 2
PG-13: 7
R: 14
